In [2]:
import matplotlib
matplotlib.use('module://ipympl.backend_nbagg')

from matplotlib import pyplot as plt
plt.ion()

import math
import numpy as np
import pickle
import sys

sys.path.append('..')
import unzipping_simulation as uzsi
from unzipping_simulation import kB

sys.path.append('../../../cn_plot_style/')
import cn_plot_style as cnps

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

np.set_printoptions(precision=3)

In [3]:
# Set the parameters of the DNA unzipping construct
with open('phage_lambda.fasta') as f:
    sequence = ''.join([line.rstrip() for line in f if not line.startswith('>')])
bases = sequence[39302:40041]
# complementary linker sequence, primer, bases, primer, hairpin (polyT left out)
bases = 'GATACGTTCTTACCCATACTCCACCGTTGC' + 'TGTGCCAACA' + 'CATTGC' + bases + 'GCAATG' + 'CAAGCTACTG' + 'CCGGTCGTAT'
nbs = 10  # 2 * 5pT
nbs_loop = 10  # hairpin with 10 pT loop
seg_a = 42588 - 42168 + 1 + 2*(8 + 10)
seg_b = 43761 - 42854 + 1 + (6 + 10) + (8 + 10)
nbp = seg_a + seg_b  # (1399)

# Set the parameters of the optical tweezers setup/assay
radius = 410e-9
r = radius
# distance between surface of the bead and the glass
h0 = 320e-9
z0 = h0
# 3D positioning
y0 = 0e-9
# A0 = attachment_point(x0, y0=y0, h0=h0, radius=radius)

# Stiffness
kappa = np.array([0.67058388e-3, 0.59549569e-3, 0.20775878e-3])
# Rotation
angles_r0 = np.array([100/180*math.pi, 45/180*math.pi])
r0_sph = np.array([radius, *angles_r0])
# lower values of k_rot decrease forces at the beginning of the unzipping
k_rot = np.array([0.1e-12*180/math.pi, 0.02e-12*180/math.pi])
#k_rot[0] = 0
#k_rot[1] = 0

T = 302.2

# Set stage displacement to be simulated
x0_min = 100e-9
x0_max = 1550e-9
resolution = 5e-9

# Set the boltzmann factor for selection of the probable states
# used to determine the simulated forces/unzipped basepairs
boltzmann_factor = 1e-5

# Set the parameters for the ssDNA model 
# Elastic modulus S, literature values are
# in the range of 0.53 ≤ K ≤ 2.2 nN 
S = 840e-12
# Persistence length 0.75 ≤ L_p ≤ 3.1 nm
L_p_ssDNA = 0.797e-9
# Contour length per base 0.43 ≤ L_0 ≤ 0.66 nm/base
# higher values stretch the unzipping curve in x
# The influence of z increases for higher numbers of unzipped basepairs
# (-> longer ssDNA -> more influence on curve)
z = 0.568e-9

# Set the parameters for the dsDNA model
pitch = 0.338e-9
L_p_dsDNA = 50e-9

# Use Nearest neighbour base-pairs for calculation of the unpairing energies?
NNBP = True
# Set the concentration of monovalent cations
c = 50e-3

In [10]:
d_angles = np.array([math.pi, math.pi])
k_rot = np.array([0, 0.03])
uzsi.F_rot(d_angles, k_rot)

array([-0.  , -0.12])

In [2]:
angles = np.linspace(-90, 90, 181)
force = uzsi.F_rot(angles*math.pi/180, 0.1e-12*180/math.pi)
energy = uzsi.E_rot(angles*math.pi/180, 0.1e-12*180/math.pi, 410e-9)
energy_pedaci = uzsi.E_rot(angles*math.pi/180, 0.1e-12*180/math.pi, 410e-9, shifted=False)

with cnps.cn_plot(context='notebook', dark=False, usetex=False) as cnp:
    fig, ax = plt.subplots()
    ax2 = cnps.second_ax(fig=fig, link_ax=ax)
    ax2.xaxis.set_visible(False)

    ax.plot(angles, force*1e12, 'c', label='Force')
    ax2.plot(angles, energy/(kB*T), 'm', label='Energy')
    #plt.legend()

    #fig.suptitle(r'Sphere rotated out of its equilibrium by angle $\Delta\theta$')
    #ax.set_title('Force and energy of a ')
    ax.set_xlabel(r'Angle $\Delta\theta$ (°)')
    ax.set_ylabel('Force (pN)')
    ax2.set_ylabel('Energy (kB*T)')
    #fig.savefig('Force_and_energy_orig.png')

FigureCanvasNbAgg()

NameError: name 'T' is not defined

In [11]:
# Calculate rotational force based on difference of r0 and r
# and plot it as tangent at the end of r
r0_theta = 180*math.pi/180
r0_phi = 0*math.pi/180
r_theta = 135*math.pi/180
r_phi = 0*math.pi/180

_k_rot = np.array([0.1e-12*180/math.pi, 0.1e-12*180/math.pi])

d_theta = r0_theta - r_theta
d_phi = r0_phi - r_phi
d_angles = np.array([d_theta, d_phi])

r0 = uzsi.sph2cart(radius*1e9, r0_theta, r0_phi)
r = uzsi.sph2cart(radius*1e9, r_theta, r_phi)

f_bead_rot_mag = uzsi.F_rot(d_angles, _k_rot)
f_bead_rot_cmp = np.r_[0, f_bead_rot_mag*1e14]
f_bead_rot = uzsi.coord_sph2cart(r_theta, r_phi, f_bead_rot_cmp, 0)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

fig = plt.figure()
ax = plt.subplot(111, projection='3d')

r0_arrow = np.c_[np.array([0,0,0]), r0]
r_arrow = np.c_[np.array([0,0,0]), r]
f_bead_rot_arrow = np.c_[r_arrow[:,1], r_arrow[:,1] + f_bead_rot]
f_bead_rot_arrow_neg = np.c_[r_arrow[:,1], r_arrow[:,1] - f_bead_rot]

# Plot scatter of points
ax.plot(*r0_arrow, lw=0)
ax.plot(*r_arrow, lw=0)
ax.plot(*f_bead_rot_arrow, lw=0)
ax.plot(*f_bead_rot_arrow_neg, lw=0)
_r0 = Arrow3D(*r0_arrow, mutation_scale=50, 
            lw=5, arrowstyle="-|>", color="k")
_r = Arrow3D(*r_arrow, mutation_scale=50, 
            lw=5, arrowstyle="-|>", color="b")
_f_bead = Arrow3D(*f_bead_rot_arrow, mutation_scale=50, 
            lw=5, arrowstyle="-|>", color="r")

_f_bead_rot = Arrow3D(*f_bead_rot_arrow_neg, mutation_scale=50, 
            lw=5, arrowstyle="-|>", color="c")

ax.add_artist(_r0)
ax.add_artist(_r)
ax.add_artist(_f_bead)
ax.add_artist(_f_bead_rot)

FigureCanvasNbAgg()

In [14]:
plt.close('all')
fig, ax = plt.subplots()
theta = np.linspace(-math.pi / 2, math.pi / 2, 1000)
phi = np.linspace(-math.pi / 2, math.pi / 2, 1000)

b = np.array(uzsi.sph2cart(1, - math.pi/2, math.pi / 4))

# theta
angle_vectors = np.array([uzsi.angle(np.array(uzsi.sph2cart(1, t, 0)), b) for t in theta])**2
ax.plot(theta*180/math.pi, angle_vectors*180/math.pi)
# phi
angle_vectors = np.array([uzsi.angle(np.array(uzsi.sph2cart(1, -math.pi/2, p)), b) for p in phi])**2
ax.plot(phi*180/math.pi, angle_vectors*180/math.pi)

ax.set_xlabel('Angle theta of vector a')
ax.set_ylabel('Angle between vectors')

FigureCanvasNbAgg()

Text(0, 0.5, 'Angle between vectors')

In [17]:
#x0 = -745.369e-9
x0 = 530e-9
A0 = uzsi.attachment_point(x0=x0, y0=y0, h0=h0, radius=radius)

f_dna = 10e-12
x_ss = uzsi.ext_ssDNA(f_dna, nbs=nbs, S=S, L_p=L_p_ssDNA, z=z, T=T)
x_ds = uzsi.ext_dsDNA_wlc(f_dna, nbp=nbp, pitch=pitch, L_p=L_p_dsDNA, T=T)

f, d, d_angles, ext_app = uzsi.F_construct_3D(A0, x_ss=x_ss, x_ds=x_ds, f_dna=f_dna,
                                              r0_sph=r0_sph, kappa=kappa, k_rot=k_rot,
                                              verbose=True, deep_verbose=False, print_result=True)
print('DNA force: {:.3f} pN, displacement: {} nm'.format(f*1e12, d*1e9))

Round 1
dna: 455.449 nm, r: 410.000 nm, c: 841.996 nm
a_dna: [ 37.007 180.   ]°, a_r: [121.451  21.995]°, a_c: [  46.683 -167.653]°, d_a_c_r_opp: [ 11.865 369.648]°, a_dna-r:153.22°
f_bead: 95.886 pN, f_bead_rot: 4.505 pN, f_cnstr_rot:4.505 pN, f_dna_total: 99.829 pN

Round 2
dna: 455.449 nm, r: 410.000 nm, c: 834.504 nm
a_dna: [ 27.525 167.223]°, a_r: [126.974  15.295]°, a_c: [  38.785 -175.627]°, d_a_c_r_opp: [ 14.24  370.922]°, a_dna-r:149.22°
f_bead: 29.509 pN, f_bead_rot: 5.117 pN, f_cnstr_rot:5.117 pN, f_dna_total: 33.598 pN

Round 3
dna: 455.449 nm, r: 410.000 nm, c: 832.830 nm
a_dna: [ 25.972 162.265]°, a_r: [128.023  12.552]°, a_c: [  37.356 -178.931]°, d_a_c_r_opp: [ 14.621 371.482]°, a_dna-r:148.39°
f_bead: 22.511 pN, f_bead_rot: 5.241 pN, f_cnstr_rot:5.241 pN, f_dna_total: 24.340 pN

Round 4
dna: 455.449 nm, r: 410.000 nm, c: 832.493 nm
a_dna: [ 25.862 160.558]°, a_r: [128.139  11.437]°, a_c: [ 37.208 179.752]°, d_a_c_r_opp: [14.654 11.686]°, a_dna-r:148.23°
f_bead: 22.654 

In [23]:
x0 = 0e-9

f_dna = 10e-12
x_ss = uzsi.ext_ssDNA(f_dna, nbs=nbs, S=S, L_p=L_p_ssDNA, z=z, T=T)
x_ds = uzsi.ext_dsDNA_wlc(f_dna, nbp=nbp, pitch=pitch, L_p=L_p_dsDNA, T=T)

ANGLES_C = []
ANGLES_R = []
F_BEAD = []
F_DNA = []
D_BEAD = []

X0 = np.linspace(-800e-9, 800e-9, 101)

for x0 in X0:

    A0 = uzsi.attachment_point(x0=x0, y0=y0, h0=h0, radius=radius)

    f_dna_total, f_bead, d, angles_c, angles_r = uzsi.F_construct_3D(A0, x_ss=x_ss, x_ds=x_ds, f_dna=f_dna,
                                                                     r0_sph=r0_sph, kappa=kappa, k_rot=k_rot,
                                                                     verbose=False, return_plus=True)
    ANGLES_C.append(angles_c)
    ANGLES_R.append(angles_r)
    F_BEAD.append(f_bead)
    F_DNA.append(f_dna_total)
    D_BEAD.append(d)

#print('e: {} nm, l: {:.2f} nm\nr: {} nm, r: {:.2f} nm \nd: {} nm \nf_bead: {} pN, f: {:.2f} pN'.format(e*1e9, np.linalg.norm(e)*1e9,
#                                                                                           R*1e9, np.linalg.norm(R)*1e9,
#                                                                                           d*1e9,
#                                                                                           f_bead*1e12, np.linalg.norm(f_bead)*1e12))
theta_c = np.array([angle[0] for angle in ANGLES_C])
phi_c = np.array([angle[1] for angle in ANGLES_C])
theta_r = np.array([angle[0] for angle in ANGLES_R])
phi_r = np.array([angle[1] for angle in ANGLES_R])
#INIT_R = np.c_[theta_r, phi_r]
#theta_r_min = np.array([min(init_r[0], r0_sph[1]) for init_r in INIT_R])
#theta_r_max =  np.array([max(init_r[0], r0_sph[1]) for init_r in INIT_R])
#phi_r_min =  np.array([min(init_r[1], r0_sph[2]) for init_r in INIT_R])
#phi_r_max =  np.array([max(init_r[1], r0_sph[2]) for init_r in INIT_R])
f_bead = np.array([np.linalg.norm(f) for f in F_BEAD])
f_dna = np.array([np.linalg.norm(f) for f in F_DNA])
f = np.array([np.linalg.norm(f) for f in F_BEAD])
d_bead = np.array([d for d in D_BEAD])

plt.close('all')
fig, ax = plt.subplots()

ax.set_ylabel('Force (pN)')
#ax.plot(X0*1e9, f_bead*1e12, '*')
ax.plot(X0*1e9, f_bead*1e12, 'm')
ax.plot(X0*1e9, f_dna*1e12, 'c')

ax2 = plt.twinx(ax)
ax2.set_ylabel('Angle (°)')
ax2.plot(X0*1e9, theta_c*180/math.pi)
ax2.plot(X0*1e9, theta_r*180/math.pi, 'o')
ax2.plot(X0*1e9, phi_c*180/math.pi)
ax2.plot(X0*1e9, phi_r*180/math.pi, 'o')

#ax2.plot(X0*1e9, d_bead*1e9)
#ax.plot(X0*1e9, theta_r_min*180/math.pi, 'b,')
#ax.plot(X0*1e9, theta_r_max*180/math.pi, 'b,')
#ax.plot(X0*1e9, phi_r_min*180/math.pi, '.')
#ax.plot(X0*1e9, phi_r_max*180/math.pi, '.')

FigureCanvasNbAgg()

In [26]:
x0 = -806e-9

A0 = uzsi.attachment_point(x0, y0=y0, h0=h0, radius=radius)

nuz = uzsi.approx_eq_nuz_rot(A0, bases=bases, nbs=nbs, nbp=nbp,
                             r0_sph=r0_sph, kappa=kappa, k_rot=k_rot,
                             S=S, L_p_ssDNA=L_p_ssDNA, z=z,
                             pitch=pitch, L_p_dsDNA=L_p_dsDNA,
                             NNBP=NNBP, c=c, T=T)

f, d, d_angles, ext_app = uzsi.F_0_3D(A0, nbs=nbs+nuz*2, S=S, L_p_ssDNA=L_p_ssDNA, z=z, T=T,
                                      nbp=nbp, pitch=pitch, L_p_dsDNA=L_p_dsDNA,
                                      r0_sph=r0_sph, kappa=kappa, k_rot=k_rot)
print('DNA force: {:.3f} pN, displacement: {} nm, rotation: {} °'.format(f*1e12, d*1e9, d_angles*180/math.pi))

#uzsi.equilibrium_xfe0_rot(A0, bases=bases, nuz=nuz, nbs=nbs, nbp=nbp, nbs_loop=nbs_loop,
#                         r0_sph=r0_sph, kappa=kappa, k_rot=k_rot,
#                         S=S, L_p_ssDNA=L_p_ssDNA, z=z,
#                         pitch=pitch, L_p_dsDNA=L_p_dsDNA,
#                         NNBP=NNBP, c=c, T=T)


r_2d = uzsi.xfe0_fast_nuz(abs(x0), bases=bases, nbs=nbs, nbp=nbp, nbs_loop=nbs_loop,
                          r=radius, z0=h0, kappa=kappa[[0,2]], 
                          S=S, L_p_ssDNA=L_p_ssDNA, z=z,
                          pitch=pitch, L_p_dsDNA=L_p_dsDNA,
                          NNBP=NNBP, c=c, T=T)
r = uzsi.xfe0_fast_nuz_rot(A0, bases=bases, nbs=nbs, nbp=nbp, nbs_loop=nbs_loop,
                           r0_sph=r0_sph, kappa=kappa, k_rot=None,
                           S=S, L_p_ssDNA=L_p_ssDNA, z=z,
                           pitch=pitch, L_p_dsDNA=L_p_dsDNA,
                           NNBP=NNBP, c=c, T=T)
r_rot = uzsi.xfe0_fast_nuz_rot(A0, bases=bases, nbs=nbs, nbp=nbp, nbs_loop=nbs_loop,
                               r0_sph=r0_sph, kappa=kappa, k_rot=k_rot,
                               S=S, L_p_ssDNA=L_p_ssDNA, z=z,
                               pitch=pitch, L_p_dsDNA=L_p_dsDNA,
                               NNBP=NNBP, c=c, T=T)

print('          r      r_rot       r_2d')
print('nuz: {:6.0f} {:9.0f}  {:9.0f}\nf_dna: {:.3f} pN {:.3f} pN {:.3f} pN\nf_lev: {:.3f} pN {:.3f} pN {:.3f} pN'.format(
    round(float(r['NUZ0_avg'])), round(float(r_rot['NUZ0_avg'])), round(float(r_2d['NUZ0_avg'])),
    float(r['F0_avg']*1e12), float(r_rot['F0_avg']*1e12), float(r_2d['F0_avg']*1e12),
    float(np.sqrt(np.sum((r['D0_avg']*kappa)**2))*1e12), float(np.sqrt(np.sum((r_rot['D0_avg']*kappa)**2))*1e12), float(np.sqrt(np.sum((r_2d['D0_avg']*kappa[[0,2]])**2))*1e12), 
    ))

DNA force: 7.902 pN, displacement: [0. 0. 0.] nm, rotation: [0. 0.] °
          r      r_rot       r_2d
nuz:    180       464        180
f_dna: 15.247 pN 7.367 pN 15.247 pN
f_lev: 15.247 pN 0.000 pN 15.247 pN


In [3]:
#plt.close('all')
# Plot energy landscape
x0 = 400e-9 + radius*0.817 - h0*0.17 - 57e-9

with cnps.cn_plot(context='notebook'):
    fig, ax, ax2 = uzsi.plot_unzip_energy_rot(x0, y0=y0, h0=h0, bases=bases, nbs=nbs, nbp=nbp, nbs_loop=nbs_loop,
                                              radius=radius, angles_r0=angles_r0, kappa=kappa, k_rot=k_rot,
                                              S=S, L_p_ssDNA=L_p_ssDNA, z=z,
                                              pitch=pitch, L_p_dsDNA=L_p_dsDNA,
                                              NNBP=NNBP, c=c, T=T,
                                              boltzmann_factor=boltzmann_factor)
#fig.savefig('Energy_number_unzipped_basepairs.png')

FigureCanvasNbAgg()